In [1]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'price-bucket-cap' # Mention the created S3 bucket name here
print("Using bucket "+ bucket)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\shiji\AppData\Local\sagemaker\sagemaker\config.yaml
Using bucket price-optimisation-sage


In [2]:
df = pd.read_csv("cleaned_retail_price.csv")
df . head()


,qty,total_price,freight_price,unit_price,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_score,customers,...,fp1,comp_2,ps2,fp2,comp_3,ps3,fp3,lag_price,product_id_encoded,product_category_name_encoded
0,1,45.95,15.100000,45.95,39,161,2,350,4.0,57,...,15.011897,215.000000,4.4,8.760000,45.95,4.0,15.100000,45.90,0,0
1,3,137.85,12.933333,45.95,39,161,2,350,4.0,61,...,14.769216,209.000000,4.4,21.322000,45.95,4.0,12.933333,45.95,0,0
2,6,275.70,14.840000,45.95,39,161,2,350,4.0,123,...,13.993833,205.000000,4.4,22.195932,45.95,4.0,14.840000,45.95,0,0
3,4,183.80,14.287500,45.95,39,161,2,350,4.0,90,...,14.656757,199.509804,4.4,19.412885,45.95,4.0,14.287500,45.95,0,0
4,2,91.90,15.100000,45.95,39,161,2,350,4.0,54,...,18.776522,163.398710,4.4,24.324687,45.95,4.0,15.100000,45.95,0,0


In [3]:
df.shape

(676, 29)

In [4]:
df . isnull().mean() * 100

qty                              0.0
total_price                      0.0
freight_price                    0.0
unit_price                       0.0
product_name_lenght              0.0
product_description_lenght       0.0
product_photos_qty               0.0
product_weight_g                 0.0
product_score                    0.0
customers                        0.0
weekday                          0.0
weekend                          0.0
holiday                          0.0
month                            0.0
year                             0.0
s                                0.0
volume                           0.0
comp_1                           0.0
ps1                              0.0
fp1                              0.0
comp_2                           0.0
ps2                              0.0
fp2                              0.0
comp_3                           0.0
ps3                              0.0
fp3                              0.0
lag_price                        0.0
p

In [5]:
features = list(df.columns)
features

['qty',
 'total_price',
 'freight_price',
 'unit_price',
 'product_name_lenght',
 'product_description_lenght',
 'product_photos_qty',
 'product_weight_g',
 'product_score',
 'customers',
 'weekday',
 'weekend',
 'holiday',
 'month',
 'year',
 's',
 'volume',
 'comp_1',
 'ps1',
 'fp1',
 'comp_2',
 'ps2',
 'fp2',
 'comp_3',
 'ps3',
 'fp3',
 'lag_price',
 'product_id_encoded',
 'product_category_name_encoded']

In [6]:
label = features.pop(-3)
label

'lag_price'

In [7]:
x = df[features]
y = df[label]

In [8]:
x . head()

,qty,total_price,freight_price,unit_price,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_score,customers,...,ps1,fp1,comp_2,ps2,fp2,comp_3,ps3,fp3,product_id_encoded,product_category_name_encoded
0,1,45.95,15.100000,45.95,39,161,2,350,4.0,57,...,3.9,15.011897,215.000000,4.4,8.760000,45.95,4.0,15.100000,0,0
1,3,137.85,12.933333,45.95,39,161,2,350,4.0,61,...,3.9,14.769216,209.000000,4.4,21.322000,45.95,4.0,12.933333,0,0
2,6,275.70,14.840000,45.95,39,161,2,350,4.0,123,...,3.9,13.993833,205.000000,4.4,22.195932,45.95,4.0,14.840000,0,0
3,4,183.80,14.287500,45.95,39,161,2,350,4.0,90,...,3.9,14.656757,199.509804,4.4,19.412885,45.95,4.0,14.287500,0,0
4,2,91.90,15.100000,45.95,39,161,2,350,4.0,54,...,3.9,18.776522,163.398710,4.4,24.324687,45.95,4.0,15.100000,0,0


In [9]:
y . head()

0    45.90
1    45.95
2    45.95
3    45.95
4    45.95
Name: lag_price, dtype: float64

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.15, random_state=0)


In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(574, 28)
(102, 28)
(574,)
(102,)


In [12]:
trainX = pd.DataFrame(X_train) 
trainX[label] = y_train

testX = pd.DataFrame(X_test) 
testX[label] = y_test

In [13]:

trainX.to_csv("train-V-1.csv",index = False) 
testX.to_csv("test-V-1.csv", index = False)

In [14]:

# send data to S3. SageMaker will take training data from s3
sk_prefix = "sagemaker/price-optimisation/sklearncontainer" 
trainpath = sess.upload_data(
            path="train-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)
testpath = sess.upload_data(
            path="test-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

In [15]:
%%writefile script.py

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np  # for np.sqrt
import sklearn
import joblib
import argparse
import os
import pandas as pd


#from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_squared_error
#import sklearn
#import joblib
#import argparse
#import os
#import pandas as pd
    
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf
    
if __name__ == "__main__":

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # Hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    features = list(train_df.columns)
    label = features.pop(-3)  # Assuming the last column is the label for regression
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ", label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
    print("Training RandomForest Model.....")
    model = RandomForestRegressor(n_estimators=args.n_estimators, random_state=args.random_state, verbose=3, n_jobs=-1)
    model.fit(X_train, y_train)
    print("Model training complete.")
    print()

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)
    print("Model persisted at " + model_path)
    print()

    # Evaluate the model using regression metrics
    y_pred_test = model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_pred_test)
    print('Test Mean Squared Error:', test_mse)

    test_mae = mean_absolute_error(y_test, y_pred_test)
    test_rmse = np.sqrt(test_mse) # RMSE is just the square root of MSE
    test_r2 = r2_score(y_test, y_pred_test)
    
    print('Evaluation Metrics for Testing Data:')
    print('------------------------------------')
    print(f'Test Mean Squared Error (MSE): {test_mse:.4f}')
    print(f'Test Mean Absolute Error (MAE): {test_mae:.4f}')
    print(f'Test Root Mean Squared Error (RMSE): {test_rmse:.4f}')
    print(f'Test R^2 Score: {test_r2:.4f}')


Overwriting script.py


In [16]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::975050105170:role/service-role/AmazonSageMaker-ExecutionRole-20240130T131692",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

In [17]:
# launch training job, with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)
# sklearn_estimator.fit({"train": datapath}, wait=True)

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2024-04-09-13-51-58-206


2024-04-09 13:51:59 Starting - Starting the training job...
2024-04-09 13:52:15 Starting - Preparing the instances for training...
2024-04-09 13:52:50 Downloading - Downloading input data...
2024-04-09 13:53:16 Downloading - Downloading the training image...
2024-04-09 13:53:56 Training - Training image download completed. Training in progress..2024-04-09 13:54:00,186 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-04-09 13:54:00,190 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-04-09 13:54:00,232 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-04-09 13:54:00,433 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-04-09 13:54:00,445 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-04-09 13:54:00,457 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-04-09 13:

In [18]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)


2024-04-09 13:54:22 Starting - Preparing the instances for training
2024-04-09 13:54:22 Downloading - Downloading the training image
2024-04-09 13:54:22 Training - Training image download completed. Training in progress.
2024-04-09 13:54:22 Uploading - Uploading generated training model
2024-04-09 13:54:22 Completed - Training job completed
Model artifact persisted at s3://sagemaker-us-east-1-975050105170/RF-custom-sklearn-2024-04-09-13-51-58-206/output/model.tar.gz


In [19]:
artifact


's3://sagemaker-us-east-1-975050105170/RF-custom-sklearn-2024-04-09-13-51-58-206/output/model.tar.gz'

In [20]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name =  model_name,
    model_data=artifact,
    role="arn:aws:iam::975050105170:role/service-role/AmazonSageMaker-ExecutionRole-20240130T131692",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

In [21]:
model_name

'Custom-sklearn-model-2024-04-09-14-10-48'

In [22]:
##Endpoints deployment
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
)

EndpointName=Custom-sklearn-model-2024-04-09-14-10-56


INFO:sagemaker:Creating model with name: Custom-sklearn-model-2024-04-09-14-10-48
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model-2024-04-09-14-10-56
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model-2024-04-09-14-10-56


------!

In [56]:
endpoint_name

'Custom-sklearn-model-2024-04-07-18-28-17'

In [57]:
testX[features][0:1].values.tolist()

[[2.0,
  220.0,
  32.23,
  110.0,
  58.0,
  1893.0,
  1.0,
  5950.0,
  4.3,
  102.0,
  23.0,
  8.0,
  3.0,
  5.0,
  2018.0,
  4.040816327,
  8000.0,
  53.41515152,
  4.1,
  10.75303571,
  53.70952381,
  4.2,
  13.51034483,
  53.05789474,
  4.1,
  12.98,
  22.0,
  5.0]]

In [58]:
print(predictor.predict(testX[features][0:1].values.tolist()))

[41.34]


In [1]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

NameError: name 'sm_boto3' is not defined

In [101]:
import boto3
import json

# Initialize a boto3 client
client = boto3.client('sagemaker-runtime', region_name='us-east-1')

# Specify your endpoint name
endpoint_name = 'Custom-sklearn-model-2024-04-02-12-10-46'

# Format your data as a JSON array
data = [
    [2.0, 220.0, 32.23, 110.0, 58.0, 1893.0, 1.0, 5950.0, 4.3, 102.0, 23.0, 8.0, 3.0, 5.0, 2018.0, 4.040816327, 8000.0, 53.41515152, 4.1, 10.75303571, 53.70952381, 4.2, 13.51034483, 53.05789474, 4.1, 12.98, 22.0, 5.0]
]
payload = json.dumps(data)

# Invoke the endpoint
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType='application/json'
)

# Parse the response
result = json.loads(response['Body'].read().decode())
print(result)


[41.34]


In [28]:
input_array = [2.0, 220.0, 32.23, 110.0, 58.0, 1893.0, 1.0, 5950.0, 4.3, 102.0, 23.0, 8.0, 3.0, 5.0, 2018.0, 4.040816327, 8000.0, 53.41515152, 4.1, 10.75303571, 53.70952381, 4.2, 13.51034483, 53.05789474, 4.1, 12.98, 22.0, 5.0]

print(predictor.predict([input_array]))



[41.34]


In [149]:
import matplotlib.pyplot as plt

def plot_feature_importances(model, features):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    plt.figure(figsize=(10, 6))
    plt.title("Feature Importances")
    plt.bar(range(len(indices)), importances[indices], color="r", align="center")
    plt.xticks(range(len(indices)), [features[i] for i in indices], rotation=90)
    plt.xlim([-1, len(indices)])
    plt.ylabel('Importance')
    plt.xlabel('Feature')
    plt.tight_layout()
    plt.savefig("feature_importances.png")
    print("[INFO] Feature importances plot saved as feature_importances.png")

def plot_actual_vs_predicted(y_test, y_pred):
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.3)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.title('Actual vs. Predicted Values')
    plt.tight_layout()
    plt.savefig("actual_vs_predicted.png")
    print("[INFO] Actual vs. Predicted plot saved as actual_vs_predicted.png")

# Ensure to call these functions after your model has been trained and evaluated
# For feature importances
plot_feature_importances(model, features)

# For actual vs predicted values plot
plot_actual_vs_predicted(y_test, y_pred_test)


NameError: name 'y_pred_test' is not defined